# Artificial Neural Network

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [10]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [12]:
print(X[:,2])

[0 0 0 ... 0 1 0]


One Hot Encoding the "Geography" column

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [41]:
print(X[4])

[0.0 0.0 1.0 850 0 43 2 125510.82 1 1 1 79084.1]


### Splitting the dataset into the Training set and Test set

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

### Feature Scaling

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [24]:
ann= tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [25]:
ann.add(tf.keras.layers.Dense(units =6,activation='relu'))

### Adding the second hidden layer

In [26]:
ann.add(tf.keras.layers.Dense(units =6,activation='relu'))

### Adding the output layer

In [27]:
ann.add(tf.keras.layers.Dense(units =1,activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [29]:
ann.compile(optimizer= 'adam',loss='binary_crossentropy',metrics= ['accuracy'])

### Training the ANN on the Training set

In [30]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 972us/step - loss: 0.5800 - accuracy: 0.7462
Epoch 2/100
250/250 [==============================] - 0s 903us/step - loss: 0.4790 - accuracy: 0.7980
Epoch 3/100
250/250 [==============================] - 0s 996us/step - loss: 0.4458 - accuracy: 0.8055
Epoch 4/100
250/250 [==============================] - 0s 944us/step - loss: 0.4285 - accuracy: 0.8120
Epoch 5/100
250/250 [==============================] - 0s 875us/step - loss: 0.4165 - accuracy: 0.8169
Epoch 6/100
250/250 [==============================] - 0s 856us/step - loss: 0.4052 - accuracy: 0.8235
Epoch 7/100
250/250 [==============================] - 0s 860us/step - loss: 0.3929 - accuracy: 0.8296
Epoch 8/100
250/250 [==============================] - 0s 937us/step - loss: 0.3803 - accuracy: 0.8394
Epoch 9/100
250/250 [==============================] - 0s 912us/step - loss: 0.3696 - accuracy: 0.8464
Epoch 10/100
250/250 [==============================] - 0s 874us/step - l

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [51]:
cust_info = [[1,0,0,600,1,40,3,60000,2,1,1,50000]]
cust_info = sc.transform(cust_info)
cust_pred = (ann.predict(cust_info) > 0.5)

In [52]:
print(cust_pred)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [65]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [66]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1514   81]
 [ 189  216]]


0.865